# Prepare the data

Divide onto train, train-valid and test

In [1]:
import shutil, os, numpy as np

import torch, torchvision

In [2]:
data_root = 'DogsVsCats/'
train_root = data_root+'train/'
# train_valid_root = data_root+'train-valid/'
test_root = data_root+'test/'

In [3]:
files = os.listdir(train_root)

#### Define torch Datasets and transforms for data

In [4]:
IMG_SIZE = (224,224)

In [5]:
train_transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize(IMG_SIZE),
    torchvision.transforms.AutoAugment(),
    torchvision.transforms.ToTensor()
])

test_transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize(IMG_SIZE),
    torchvision.transforms.ToTensor()
])

train_dataset = torchvision.datasets.ImageFolder(train_root, transform=train_transform)
# train_valid_dataset = torchvision.datasets.ImageFolder(train_valid_root, transform=train_transform)
test_dataset = torchvision.datasets.ImageFolder(test_root, transform=test_transform)

#### Make loaders

In [6]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True, pin_memory=True)

# train_valid_loader = torch.utils.data.DataLoader(train_valid_dataset, batch_size=32, shuffle=True, pin_memory=True)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False, pin_memory=True)

# Fine-tuning

In [7]:
import torch, torchvision
from torchvision.models import resnet18 as model, ResNet18_Weights as model_weights


from tqdm import tqdm
# resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)

In [8]:
class finalLayer(torch.nn.Module):
    def __init__(self, features_in):
        super(finalLayer, self).__init__()
        self.linear = torch.nn.Linear(features_in, 1)
        # self.RELU = torch.nn.ReLU(inplace=True)
        self.sigm = torch.nn.Sigmoid()

    def forward(self, x):
        x = self.sigm(self.linear(x))
        return x


def my_resnet():
    resnet = model(weights=model_weights.IMAGENET1K_V1)

    resnet.fc = finalLayer(resnet.fc.in_features)
    return resnet

my_net = my_resnet()

In [9]:
my_net

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [10]:
params = [p for p in my_net.parameters() if p.requires_grad]
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.NAdam(
    params,
    lr=0.01,
    weight_decay=0.0005
)

lr_scheduler = torch.optim.lr_scheduler.StepLR(
    optimizer,
    step_size=3,
    gamma=0.1
)

In [13]:
def train_one_epoch(model, optimizer, criterion, data_loader, scheduler, epoch, freq_print):
    print(f"Training epoch {epoch}")
    model.train()
    for batch_idx, (X, y) in tqdm(enumerate(data_loader), total=int(len(data_loader.dataset) / data_loader.batch_size)):
        y = y.flatten().float()
        yhat = model(X).flatten()
        loss = criterion(yhat, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch_idx %freq_print == 0:
            acc = ((yhat > .5) == y).sum() / data_loader.batch_size
            print(f'Batch n {batch_idx}')
            print(f'Loss: {loss.item():.4f} Accuracy: {acc:.4f} ')

    if scheduler is not None:
        scheduler.step()

# Тренируем модель

- СИТУАЦИЯ:

- Тренировал модели весь день, пробовал разные вариации - ResNet18, 32, 50, googlenet.

    - Глубина файн-тюнинга - 5 эпох. Все из перечисленных моделей выдавали LogLoss на уровне примерно .69

- МОИ МЫСЛИ:
- У меня есть предположение, что мой код пытался тренировать все слои модели, поскольку батчи учились довольно долго (по сравнению с Керас моделью из второго ноутбука, который приложил)

- ВОПРОС:
- Подскажите, пожалуйста, в чём мои ошибки? При файн-тюнинге пользовался:
    - https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html
    - https://www.kaggle.com/code/francescolorenzo/96-fine-tuning-resnet34-with-pytorch#Generating-Predictions-

In [12]:
num_epochs = 10

for epoch in range(num_epochs):
    train_one_epoch(my_net, optimizer, criterion, train_loader, lr_scheduler, epoch, 50)

Training epoch 0


  0%|▏                                                                                 | 1/390 [00:04<32:18,  4.98s/it]

Batch n 0
Loss: 0.7853 Accuracy: 0.4688 


 13%|██████████▌                                                                      | 51/390 [04:08<27:19,  4.84s/it]

Batch n 50
Loss: 0.6946 Accuracy: 0.6250 


...

Batch n 100
Loss: 0.6929 Accuracy: 0.5312 


 39%|██████████████████████████████▉                                                 | 151/390 [12:58<21:16,  5.34s/it]

Batch n 150
Loss: 0.6949 Accuracy: 0.4844 


 46%|████████████████████████████████████▌                                           | 178/390 [15:27<18:24,  5.21s/it]


KeyboardInterrupt: 

In [ ]:
def make_submission(model):
    # sigm = torch.nn.Sigmoid()
    result = []
    model.eval()
    with torch.no_grad():
        for x, _ in tqdm(test_loader):
            output = model(x).flatten().tolist()
            result += output

    return result

In [ ]:
labels = make_submission(my_net)

# Make submission

In [ ]:
import pandas as pd

In [ ]:
res_df = pd.read_csv('sample_submission.csv')

In [ ]:
res_df['label'] = labels

In [ ]:
res_df.to_csv('submission.csv', index=False)